# Project: Traffic Simulation Model


## Introduction

This project implements a traffic simulation model using the Mesa framework. The simulation aims to model the movement of cars within a city grid, incorporating traffic lights and parking spots. The primary goal is to simulate realistic traffic behavior, including car movements, traffic light interactions, and parking dynamics.

The simulation is built on a 24x24 grid, where each cell can represent different elements such as roads, buildings, parking spots, and traffic lights. Car agents navigate through the grid based on predefined directional coordinates and traffic light states, aiming to reach their target parking spots.

Key features of the project include:

- Car Agents: Simulated cars that move around the grid, following traffic rules and aiming to park in available spots.
- Traffic Light Agents: Traffic lights that control the flow of car agents at intersections, switching between red and green states.
- Grid Representation: A multi-layer grid that includes buildings, roads, and parking spots.
- Visualization: Tools for visualizing the simulation, including the positions of car agents and the states of traffic lights.

The project leverages the Mesa framework for agent-based modeling and includes various components for data collection and visualization to analyze the simulation results.

## Installation



To set up the project, you need to install several dependencies. These libraries are essential for running the traffic simulation model and visualizing the results.

### Required Libraries

- **Mesa (version 3.1.0.dev)**: A Python framework for agent-based modeling. It provides the core functionality for creating and managing agents, as well as tools for visualization and data collection.
- **Seaborn**: A data visualization library based on Matplotlib. It is used for creating informative and attractive statistical graphics.
- **Solara**: A library for building interactive web applications in Python. It is used for creating interactive visualizations of the simulation.
- **Pandas**: A powerful data manipulation and analysis library. It is used for handling and analyzing data within the simulation.
- **NumPy**: A fundamental package for scientific computing with Python. It is used for numerical operations and handling arrays.
- **Random**: A module for generating random numbers. It is used for randomizing agent behaviors and movements.
- **Time**: A module for time-related functions. It is used for managing simulation timing and delays.

### Installation Steps

To install the required libraries, you can use the following commands:
```python
#%pip install --quiet --upgrade mesa==3.1.0.dev
#%pip install solara
#%pip install seaborn
#%pip install pandas
#%pip install numpy

In [1]:

import time
# Data visualization tools.
import seaborn as sns
import random
import numpy as np

# Data manipulation and analysis.
import pandas as pd

#Mesa library
import mesa
from mesa.visualization.utils import update_counter
from mesa.visualization import SolaraViz, make_plot_component, make_space_component
from mesa.datacollection import DataCollector
print(f"Mesa version: {mesa.__version__}")


Mesa version: 3.1.0.dev


## Map Construction



In this section, we will define the structure and layout of the simulation environment. The map construction involves setting up a grid that represents the city, including buildings, parking spots, directional coordinates, and traffic lights. Each of these elements plays a crucial role in the simulation:

- **Buildings**: Represent static structures within the city grid. They serve as obstacles that car agents must navigate around.
- **Parking Spots**: Designated areas where car agents aim to park. These spots are the primary destinations for the car agents.
- **Directional Coordinates**: Predefined paths that guide the movement of car agents through the grid. These coordinates help simulate realistic traffic flow.
- **Traffic Lights**: Control the flow of car agents at intersections. They switch between red and green states to manage traffic and prevent collisions.

We will use the Mesa framework to create and manage the grid, ensuring that all elements are correctly placed and interact as expected. The map construction is a crucial step in creating a realistic and functional traffic simulation model.

#### Buildings

In [2]:
from mapBuild.buildings import buildings_coords

#### Parking spots

In [3]:
from mapBuild.parkingSpots import parking_spots

#### Direccional coords(Left, Right, Down, Up)

In [4]:
from mapBuild.leftCoords import left_coords
from mapBuild.rightCoords import right_coords
from mapBuild.upCoords import up_coords
from mapBuild.downCoords import down_coords

#### Traffic Lights

In [5]:
from mapBuild.trafficLights import traffic_light_coords
print(traffic_light_coords)

[[[6, 21], [7, 21], 'red'], [[8, 22], [8, 23], 'green'], [[6, 16], [7, 16], 'red'], [[8, 18], [8, 17], 'green'], [[0, 6], [1, 6], 'red'], [[2, 4], [2, 5], 'green'], [[5, 0], [5, 1], 'red'], [[6, 2], [7, 2], 'green'], [[17, 8], [17, 9], 'red'], [[18, 7], [19, 7], 'green']]


## Traffic Model

In this section, we will explain the traffic simulation model, its components, and main functionalities. The model is designed to simulate the movement of car agents within a city grid, incorporating traffic lights and parking spots to create a realistic traffic environment.

### Components

1. **Car Agents**:
   - **Purpose**: Simulate individual cars moving through the city grid.
   - **Behavior**: Car agents navigate the grid based on directional coordinates, obey traffic lights, and aim to park in available parking spots.

2. **Traffic Light Agents**:
   - **Purpose**: Control the flow of car agents at intersections.
   - **Behavior**: Traffic lights switch between red and green states at regular intervals, managing the movement of car agents and preventing collisions.

3. **Grid**:
   - **Purpose**: Represent the city layout, including roads, buildings, parking spots, and traffic lights.
   - **Structure**: A 24x24 grid where each cell can represent different elements such as roads, buildings, parking spots, and traffic lights.

### Main Functionalities

1. **Initialization**:
   - The model initializes the grid, placing buildings, parking spots, directional coordinates, and traffic lights in their respective positions.
   - Car agents are created and placed on the grid, ready to navigate the city.

2. **Agent Movement**:
   - Car agents move through the grid based on predefined directional coordinates.
   - They obey traffic lights and navigate around buildings to reach their target parking spots.

3. **Traffic Light Control**:
   - Traffic light agents manage the flow of car agents at intersections.
   - They switch between red and green states at regular intervals, ensuring smooth traffic flow and preventing collisions.

4. **Parking Dynamics**:
   - Car agents aim to park in available parking spots.
   - The model tracks the occupancy of parking spots and updates the status of car agents accordingly.

This comprehensive model allows for the simulation of realistic traffic behavior, providing valuable insights into traffic dynamics and potential improvements for urban planning.

In [6]:
class TrafficModel(mesa.Model):
    def __init__(
        self,
        width=24,
        height=24,
        num_agents=1,
        left_coords=None,
        right_coords=None,
        up_coords=None,
        down_coords=None,
        buildings_coords=None,
        parking_coords=None,
        traffic_light_coords=None,
    ):
        super().__init__()
        #ATTRIBUTES OF THE MODEL----------------------------------------------------------------------------------
        self.random = random.Random()
        self.steps = 0

        #Parameters
        self.width = width
        self.height = height
        self.num_agents = num_agents
        self.parkings_coords = parking_coords
        self.traffic_light_coords = traffic_light_coords

        # Dictionary to store directions for each cell
        self.directions = {}

        # Global map to store the positions of all agents at each step
        self.global_map = {}
        
        #Create a dictionary mapping each parking spot to a unique key starting from 1
        self.ParkingSpots = {i + 1: spot for i, spot in enumerate(parking_coords)}
        
        #LAYERS OF THE GRID---------------------------------------
        #Layer for buildings
        buildingLayer = mesa.space.PropertyLayer(
            "building", width, height, np.int64(0), np.int64
        )
        #layer for parking spots
        parkingsLayer = mesa.space.PropertyLayer(
            "parking", width, height, np.int64(0), np.int64
        )

        # Set the to layer to its corresponding cell
        self.set_building_cells(buildings_coords, buildingLayer)
        self.set_parking_cells(parking_coords, parkingsLayer)
    
        # Create a MultiGrid object the two layers
        self.grid = mesa.space.MultiGrid(
            width, height, True, (buildingLayer, parkingsLayer)
        )

        #CREATION OF AGENTS IN THE GRID---------------------------------------    
        # Initialize the allowed directions for each cell
        self.initialize_directions(left_coords, right_coords, up_coords, down_coords)

        # Create the CarAgents and place them on the grid
        self.create_CarAgents()

        # Place the traffic lights on the grid
        self.place_TrafficLight_agents()

        # Initialize the DataCollector
        self.datacollector = mesa.DataCollector(
            model_reporters={},
            agent_reporters={
                "TargetParkingSpot": lambda agent: (
                    agent.target_parking_spot if isinstance(agent, CarAgent) else None
                )
            },
        )

        # Collect initial data
        self.datacollector.collect(self)


    #INITIALIZER METHODS----------------------------------------------------------------------------------
    # Initialize allowed directions for each cell in the grid
    def initialize_directions(self, left_coords, right_coords, up_coords, down_coords):
        for x in range(self.width):
            for y in range(self.height):
                self.directions[(x, y)] = {
                    "left": False,
                    "right": False,
                    "up": False,
                    "down": False,
                }

        # Set specific directions for each list
        for coord in left_coords:
            if coord in self.directions:
                self.directions[coord]["left"] = True

        for coord in right_coords:
            if coord in self.directions:
                self.directions[coord]["right"] = True

        for coord in up_coords:
            if coord in self.directions:
                self.directions[coord]["up"] = True

        for coord in down_coords:
            if coord in self.directions:
                self.directions[coord]["down"] = True


     # Create agents and place them on the grid
    def create_CarAgents(self):
        used_parking_spots = set()

        for i in range(self.num_agents):
            available_spawn_spots = [
                spot
                for spot in self.ParkingSpots.values()
                if spot not in used_parking_spots
            ]

            if available_spawn_spots:
                Spawn = random.choice(available_spawn_spots)
            else:
                # If no available spots for spawning, take some action (e.g., break or similar)
                print("No available spots for spawn")
                break

            used_parking_spots.add(Spawn)

            # Create a list of available spots for the target parking (excluding the spawn spot and used spots)
            available_target_spots = [
                spot
                for spot in self.ParkingSpots.values()
                if spot != Spawn and spot not in used_parking_spots
            ]

            if available_target_spots:
                target_parking_spot = random.choice(available_target_spots)
            else:
                # If no available spots for target parking, take some action (e.g., break or continue)
                print("No available spots for target parking")
                break  # or continue with some other action.

            # Mark the target spot as used
            used_parking_spots.add(target_parking_spot)

            print(f"Spawn: ({Spawn}), Target: ({target_parking_spot})")

            agent = CarAgent(
                self, Spawn, target_parking_spot
            )  # Pass the coordinates

            # Place the agent in the 'Spawn' cell using the correct coordinates
            self.grid.place_agent(agent, Spawn)













    # Place traffic light agents on the grid
# Place traffic light agents on the grid
    def place_TrafficLight_agents(self):
        for idx, traffic_light_info in enumerate(self.traffic_light_coords):
            traffic_light_id = f"sema_{idx}"  # Unique ID for each traffic light
            positions = traffic_light_info[:2]  # Assuming the positions are the first two values
            initial_state = traffic_light_info[2]  # The initial state is the third value

            # Create the traffic light agent for each position
            for pos in positions:
                # Ensure the position is passed when creating the agent
                sema_agent = TrafficLightAgent(
                    traffic_light_id, initial_state, self
                )  # Assign the ID, state, model, and position

                # Place the agent on the grid
                self.grid.place_agent(sema_agent, pos)

                # Print information about the placed agent
                print(f"Placed TrafficLightAgent: ID={sema_agent.unique_id}, State={sema_agent.state}, Position={sema_agent.pos}")
                
                















                
                
                
    #GETTER METHODS----------------------------------------------------------------------------------
    # Fetch the direction info for a specific cell
    def get_cell_directions(self, pos):    
        return self.directions.get(pos, None)

    # Create a global map of the current state of the simulation
    def get_global_map(self):
        # Add the current step and agents' positions to the global_map
        self.global_map = {
            "Cars": [],
            "Traffic_Lights": {}
        }

        # Collect all CarAgents and TrafficLightAgents and sort them by unique_id
        car_agents = []
        trafficLight = {}

        for contents, (x, y) in self.grid.coord_iter():
            for agent in contents:
                if isinstance(agent, CarAgent):
                    car_agents.append(agent)
                elif isinstance(agent, TrafficLightAgent):
                    trafficLight[agent.unique_id] = agent.state == "green"

        car_agents.sort(key=lambda agent: agent.unique_id)

        # Append the positions of the sorted CarAgents to the global_map
        for agent in car_agents:
            self.global_map["Cars"].append(agent.pos)

        # Append the traffic light states to the global_map
        self.global_map["Traffic_Lights"] = trafficLight
        print(self.global_map)
    
        
    # LAYER METHODS----------------------------------------------------------------------------------
    # Set the value of cells to indicate buildings
    def set_building_cells(self, buildings_coords, buildingLayer):
        for coord in buildings_coords:
            buildingLayer.set_cell(coord, 1)

    # Set the value of cells to indicate parking spots
    def set_parking_cells(self, parking_coords, parkingsLayer):

        for coord in parking_coords:
            parkingsLayer.set_cell(coord, 1)

    # STEP METHOD----------------------------------------------------------------------------------
    # Execute one step of the model, shuffle agents, and collect data
    def step(self):
        # Shuffle and execute the step method for all agents
        self.agents.shuffle_do("step")
        # Collect data for the current step
        self.datacollector.collect(self)
        # Create a global map of the current state
        self.get_global_map()


## Car Agent
Main points of the agent's functionality:

- Initialization:
Each agent has a unique ID and is created with the model. 
It is randomly positioned on the model's grid.
- Movement:
The main movement method is `move()`.
This method is called at each model step `step()`.
- Directions:
A dictionary of directions is used: "up," "down," "left," "right".
- Choosing a random direction:
A random direction is selected from those allowed in the current cell.
- Checking for an empty space:
Before moving, it checks if the new space is empty.

**Basic Configuration:**
- Defines the width and height of the grid (default is 24x24). 
Sets the number of agents (default is 10).
- Path Customization:
Allows defining custom paths using lists of coordinates (x, y) for each direction (up, down, left, right).
- Model Initialization: 
Creates a two-dimensional grid (MultiGrid). 
Initializes the scheduler to manage model steps.
- Agent Creation: 
Randomly places agents on the grid.
- Direction Initialization: 
Uses the initialize_directions() method to set up allowed directions in each cell based on the provided lists.
- Model Steps: The step() method is called at each step to update the state of all agents.

In [7]:
class CarAgent(mesa.Agent):
    def __init__(self, model, spawn_position, target_parking_spot):
        super().__init__(model)
        self.active = True
        self.parking_spots = [coord for coord in model.parkings_coords if coord != spawn_position]
        self.distance_travelled = 0
        self.target_parking_spot = random.choice(list(model.ParkingSpots.values()))

    def check_semaphore(self, current_position): #for future implementation, check if the agent is a car
        agents_at_position = self.model.grid.get_cell_list_contents([current_position])
        semaphore_agent = None
        for agent in agents_at_position:
            if agent.__class__ == TrafficLightAgent:
                semaphore_agent = agent
        if semaphore_agent is None:
            return True
        else:
            if semaphore_agent.state == "red":
                print(f"Semaphore at {current_position} is red; agent cannot move.")
                return False
            elif semaphore_agent.state == "green":
                print(f"Semaphore at {current_position} is green; agent can move.")
                return True
            
    def check_agent(self, new_position):
        agents_at_position = self.model.grid.get_cell_list_contents([new_position])
        car_agent = None
        for agent in agents_at_position:
            if agent.__class__ == CarAgent:
                car_agent = agent
        if car_agent is None: 
            return True
        else:
            return False

    def move(self):
# Get current position and allowed directions
        current_position = self.pos
        possible_current_directions = self.model.get_cell_directions(current_position)
        
        # Check for any available directions in this cell
        if not possible_current_directions:
            print(f"No directions available for agent at position {current_position}")
            return

        # Filter allowed directions and select one randomly
        possible_directions = [direction for direction, allowed in possible_current_directions.items() if allowed]
        if not possible_directions:
            print(f"No movement options for agent at position {current_position}")
            return

        # Checks if it can move acccording to the sempahore
        if not self.check_semaphore(current_position):
            return 

        direction = random.choice(possible_directions)

        # Calculate the new position based on the chosen direction
        dx, dy = {
            "up": (0, 1),
            "down": (0, -1),
            "left": (-1, 0),
            "right": (1, 0)
        }[direction]

        new_position = (self.pos[0] + dx, self.pos[1] + dy)
        if not self.check_agent(new_position):
            return 
        
        self.model.grid.move_agent(self, new_position)
        self.distance_travelled += 1  # Increment distance traveled
        self.pos = new_position       # Update the position
            
    def move_to_target(self):
        # Moverse hasta alcanzar el destino
        while self.active:
            if self.pos in self.parking_spots and self.distance_travelled > 0:
                self.active = False
                break
            else:
                moved = self.move()
                if not moved:  # Si no puede moverse, detener el bucle
                    break            
            
    def step(self):
        self.move_to_target()

## Traffic Light Agent

In [8]:
class TrafficLightAgent(mesa.Agent):
    def __init__(self, unique_id, state, model):
        super().__init__( model)
        self.unique_id = unique_id
        self.state = state  # Estado del semáforo ("red", "yellow", "green")
        self.time_counter = 0

    def change_state(self):
        """ Cambia el estado del semáforo entre rojo, amarillo y verde """
        if self.state == "red":
            self.state = "yellow"  # Rojo a amarillo
        elif self.state == "yellow":
            self.state = "green"  # Amarillo a verde
        else:
            self.state = "red"  # Verde a rojo


    def communicate_with_neighbors(self):
        """Lógica de comunicación con semáforos cercanos"""
        # Obtener los vecinos en la grilla
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        
        # Lista para almacenar los IDs o estados de los vecinos con los que se comunica
        communicated_neighbors = []

        # Comunicarse con los semáforos vecinos
        for neighbor in neighbors:
            if isinstance(neighbor, TrafficLightAgent):  # Verificar si es un semáforo vecino
                communicated_neighbors.append((neighbor.unique_id, neighbor.state, neighbor.pos))
                '''
                if neighbor.unique_id!= self.unique_id:
                    print("funcionalidad diferente")
                    print("Vecino",  neighbor.unique_id)
                    if neighbor.state == "green":  # Si el vecino está en verde
                        self.state = "red"
                    elif neighbor.state == "red":
                        self.state = "green"
                else:
                    if neighbor.state == "red":  # Si el vecino está en verde
                        self.state = "red"
                    elif neighbor.state == "green":
                        self.state = "green"

                 
                if neighbor.state == "green":  # Si el vecino está en verde
                    self.state = "green"
                elif neighbor.state == "red":
                    self.state = "red"
                '''
        if len(communicated_neighbors) == 2: #change color
            first_neighbor = communicated_neighbors[0]
            second_neighbor = communicated_neighbors[1]

            if first_neighbor[0] == self.unique_id:
                self.state = second_neighbor[1]
                second_neighbor.state = self.state
            elif communicated_neighbors[0][0] != self.unique_id:
                self.state = first_neighbor[1]
                first_neighbor.state = self.state
        else: # esquina externa (mimic)
            neighbor_state = communicated_neighbors[0][1]
            if neighbor_state == "green":  # Si el vecino está en verde
                self.state = "green"
            elif neighbor_state == "red":
                self.state = "red"
            communicated_neighbors[0][2] = self.state
        
        '''

        x2, y2 = self.pos
        #print("Coordinates 1", x2, y2)
        x1, y1 = communicated_neighbors[0][2]
        #print("Coordinates 2", x1, y1)
        
        if len(communicated_neighbors) == 2: #esquina interna
            if communicated_neighbors[0][0] == self.unique_id: #mimic color
                if x1 == x2: #they are different in y
                    if y1 > y2: # that's when they change
                        if communicated_neighbors[0][1] == "green":  # Si el vecino está en verde
                            self.state = "green"
                        elif communicated_neighbors[0][1] == "red":
                            self.state = "red"
                else: #different in x
                    if x1 > x2: # that's when they change
                        if communicated_neighbors[0][1] == "green":  # Si el vecino está en verde
                            self.state = "green"
                        elif communicated_neighbors[0][1] == "red":
                            self.state = "red"
            else: #opposite color
                if x1 == x2: 
                    if y1 > y2:
                        print("Change supposed to happen")
                        if communicated_neighbors[0][1] == "green":  # Si el vecino está en verde
                            self.state = "red"
                        elif communicated_neighbors[0][1] == "red":
                            self.state = "green"
                else: 
                    if x1 > x2:
                        print("Change supposed to happen")
                        if communicated_neighbors[0][1] == "green":  # Si el vecino está en verde
                            self.state = "red"
                        elif communicated_neighbors[0][1] == "red":
                            self.state = "green"
        else: # esquina externa
            if communicated_neighbors[0][1] == "green":  # Si el vecino está en verde
                self.state = "green"
            elif communicated_neighbors[0][1] == "red":
                self.state = "red"
        
        '''

        # Mostrar información de los vecinos con los que se comunicó
        print(f"TrafficLight {self.unique_id} with position: {self.pos} communicated with neighbors: {communicated_neighbors}")


    def step(self):
        """Actualizar el estado y comunicarse con vecinos en cada paso"""
        self.time_counter += 1
        if self.time_counter >= 5:
            print("Entrando")
            #self.change_state()
            self.communicate_with_neighbors()
            self.time_counter = 0
        

## Model Deployment 

In [9]:
# Crear una instancia de TrafficModel
model = TrafficModel(24, 24, 5, left_coords, right_coords, up_coords, down_coords, buildings_coords, parking_spots, traffic_light_coords)

# Crear una matriz para contar los agentes de tipo CarAgent
agent_counts = np.zeros((model.grid.width, model.grid.height))

# Imprimir las coordenadas de los agentes
print("Coordenadas de los agentes:")
for agent in model.agents:  # Itera sobre todos los agentes en el modelo
    if isinstance(agent, CarAgent):  # Verifica si es un CarAgent
        print(f"Agente en posición: {agent.pos}")



Spawn: ((17, 6)), Target: ((17, 21))
Spawn: ((10, 7)), Target: ((10, 12))
Spawn: ((4, 12)), Target: ((3, 6))
Spawn: ((17, 4)), Target: ((5, 17))
Spawn: ((2, 14)), Target: ((10, 19))
Placed TrafficLightAgent: ID=sema_0, State=red, Position=[6, 21]
Placed TrafficLightAgent: ID=sema_0, State=red, Position=[7, 21]
Placed TrafficLightAgent: ID=sema_1, State=green, Position=[8, 22]
Placed TrafficLightAgent: ID=sema_1, State=green, Position=[8, 23]
Placed TrafficLightAgent: ID=sema_2, State=red, Position=[6, 16]
Placed TrafficLightAgent: ID=sema_2, State=red, Position=[7, 16]
Placed TrafficLightAgent: ID=sema_3, State=green, Position=[8, 18]
Placed TrafficLightAgent: ID=sema_3, State=green, Position=[8, 17]
Placed TrafficLightAgent: ID=sema_4, State=red, Position=[0, 6]
Placed TrafficLightAgent: ID=sema_4, State=red, Position=[1, 6]
Placed TrafficLightAgent: ID=sema_5, State=green, Position=[2, 4]
Placed TrafficLightAgent: ID=sema_5, State=green, Position=[2, 5]
Placed TrafficLightAgent: ID=s

/var/folders/0b/36rw0ltd0vg3t22nhv3dkq0r0000gn/T/ipykernel_29274/1665195651.py:38: FutureWarning: The new PropertyLayer and _PropertyGrid classes experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1932
  buildingLayer = mesa.space.PropertyLayer(


In [10]:
for i in range(5):  # Eje    cutar el modelo por 10 pasos
  model.step()

{'Cars': [(18, 6), (10, 8), (4, 11), (18, 4), (1, 14)], 'Traffic_Lights': {'sema_4': False, 'sema_5': True, 'sema_6': False, 'sema_7': True, 'sema_2': False, 'sema_0': False, 'sema_3': True, 'sema_1': True, 'sema_8': False, 'sema_9': True}}
{'Cars': [(18, 7), (11, 8), (4, 12), (17, 4), (1, 13)], 'Traffic_Lights': {'sema_4': False, 'sema_5': True, 'sema_6': False, 'sema_7': True, 'sema_2': False, 'sema_0': False, 'sema_3': True, 'sema_1': True, 'sema_8': False, 'sema_9': True}}
Semaphore at (18, 7) is green; agent can move.
{'Cars': [(18, 8), (12, 8), (4, 11), (18, 4), (1, 12)], 'Traffic_Lights': {'sema_4': False, 'sema_5': True, 'sema_6': False, 'sema_7': True, 'sema_2': False, 'sema_0': False, 'sema_3': True, 'sema_1': True, 'sema_8': False, 'sema_9': True}}
{'Cars': [(19, 8), (12, 7), (4, 12), (17, 4), (1, 11)], 'Traffic_Lights': {'sema_4': False, 'sema_5': True, 'sema_6': False, 'sema_7': True, 'sema_2': False, 'sema_0': False, 'sema_3': True, 'sema_1': True, 'sema_8': False, 'sema_

TypeError: unhashable type: 'list'

# Visualization 1

In [ ]:
#%pip install solara

In [ ]:
import solara


Define the agent portrayal function

In [ ]:
def agent_portrayal(agent):
    size = 20
    color = "tab:red"
    shape = "circle"
    
    # Verificar el tipo de agente
    if isinstance(agent, CarAgent):
        size = 50
        color = "tab:blue"
        shape = "circle"
    elif isinstance(agent, TrafficLightAgent):
        size = 20
        shape = "circle"
        if agent.state == "red":
            color = "tab:red"
        elif agent.state == "green":
            color = "tab:green"
        elif agent.state == "yellow":  # Agregar el estado "yellow" para semáforos
            color = "yellow"  # O cualquier otro color que prefieras para el estado amarillo
    return {"size": size, "color": color, "shape": shape}


 Define the model parameters

In [ ]:
model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 10,
        "label": "Number of agents:",
    },
    "width": 24,
    "height": 24,
}

Create the initial model instance

In [ ]:
initial_model = TrafficModel(
    num_agents=7,
    width=24,
    height=24,
    left_coords= left_coords,
    right_coords=right_coords,
    up_coords=up_coords,
    down_coords=down_coords,
    buildings_coords=buildings_coords,
    parking_coords=parking_spots,
    traffic_light_coords=traffic_light_coords
)



Spawn: ((20, 18)), Target: ((4, 3))
Spawn: ((5, 17)), Target: ((8, 15))
Spawn: ((10, 12)), Target: ((2, 14))
Spawn: ((3, 6)), Target: ((10, 7))
Spawn: ((9, 2)), Target: ((17, 4))
Spawn: ((20, 4)), Target: ((10, 19))
Spawn: ((20, 15)), Target: ((17, 21))
Placed TrafficLightAgent: ID=sema_0, State=red, Position=(6, 21)
Placed TrafficLightAgent: ID=sema_0, State=red, Position=(7, 21)
Placed TrafficLightAgent: ID=sema_1, State=green, Position=(8, 22)
Placed TrafficLightAgent: ID=sema_1, State=green, Position=(8, 23)
Placed TrafficLightAgent: ID=sema_2, State=red, Position=(6, 16)
Placed TrafficLightAgent: ID=sema_2, State=red, Position=(7, 16)
Placed TrafficLightAgent: ID=sema_3, State=green, Position=(8, 18)
Placed TrafficLightAgent: ID=sema_3, State=green, Position=(8, 17)
Placed TrafficLightAgent: ID=sema_4, State=red, Position=(0, 6)
Placed TrafficLightAgent: ID=sema_4, State=red, Position=(1, 6)
Placed TrafficLightAgent: ID=sema_5, State=green, Position=(2, 4)
Placed TrafficLightAgent

Create the visualization components

In [ ]:
propertylayer_portrayal={"building": {"color":"blue","colorbar":False}, "parking":{"color":"yellow","colorbar":False}}
SpaceGraph = make_space_component(agent_portrayal, propertylayer_portrayal=propertylayer_portrayal)

Create the SolaraViz page

In [ ]:
page = SolaraViz(
    initial_model,
    components=[SpaceGraph],
    model_params=model_params,
    name="Traffic Simulation",
)
page

NameError: name 'initial_model' is not defined